In [1]:
import pandas as pd

In [27]:
# load
data_df = pd.read_csv('datasets/mood_preprocessed_aggr_per_day.csv')
data_df['date'] = pd.to_datetime(data_df['date'])
data_df

,date,id,screen,call,sms,appCat.builtin,appCat.communication,appCat.entertainment,appCat.finance,appCat.game,...,appCat.other,appCat.social,appCat.travel,appCat.unknown,appCat.utilities,appCat.weather,circumplex.arousal,circumplex.valence,activity,mood
0,2014-02-17,AS14.01,0.000000,2.0,0.0,0.000,0.000,0.0,0.0,0.0,...,0.000,0.000,0.000,0.0,0.000,0.0,-1.000000,0.0,0.071429,6.0
1,2014-02-17,AS14.02,0.000000,4.0,2.0,0.000,0.000,0.0,0.0,0.0,...,0.000,0.000,0.000,0.0,0.000,0.0,0.000000,1.0,0.142857,7.0
2,2014-02-17,AS14.03,0.000000,0.0,1.0,0.000,0.000,0.0,0.0,0.0,...,0.000,0.000,0.000,0.0,0.000,0.0,1.000000,2.0,0.290000,9.0
3,2014-02-17,AS14.06,0.000000,5.0,0.0,0.000,0.000,0.0,0.0,0.0,...,0.000,0.000,0.000,0.0,0.000,0.0,-1.000000,0.0,0.120000,7.0
4,2014-02-17,AS14.08,0.000000,0.0,4.0,0.000,0.000,0.0,0.0,0.0,...,0.000,0.000,0.000,0.0,0.000,0.0,0.000000,1.0,0.000000,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1968,2014-06-05,AS14.24,1620.056999,11.0,0.0,801.994,795.073,0.0,0.0,0.0,...,47.360,171.964,225.737,0.0,0.000,0.0,0.223077,1.0,0.253627,7.0
1969,2014-06-06,AS14.24,2169.781000,16.0,0.0,838.897,1205.799,0.0,0.0,0.0,...,36.414,0.000,0.000,0.0,2.072,0.0,0.702128,1.0,0.287322,7.0
1970,2014-06-07,AS14.24,1121.516999,0.0,2.0,171.285,411.638,0.0,0.0,0.0,...,6.214,106.850,0.000,0.0,0.000,0.0,0.333333,1.0,0.317301,8.0
1971,2014-06-08,AS14.24,111.459000,0.0,0.0,88.239,3.042,0.0,0.0,0.0,...,18.069,0.000,0.000,0.0,0.000,0.0,-1.409091,1.0,0.102054,7.0


In [28]:
def create_user_specific_windows_and_targets(data, target_column, window_size):
    windows = []
    targets = []
    
    # Iterate over each unique user to ensure windows are user-specific
    for user in data['id'].unique():
        user_data = data[data['id'] == user]
        
        for i in range(len(user_data) - window_size):
            end_ix = i + window_size
            
            # Ensure we do not go out of bounds when accessing the target
            if end_ix >= len(user_data):
                break

            # Extract the window and target, excluding the target column from the window
            window = user_data.iloc[i:end_ix].drop(columns=[target_column, 'id']).values.tolist()
            target = user_data.iloc[end_ix][target_column]
            windows.append(window)
            targets.append(target)

    return windows, targets



In [29]:
# window size
n = 5 

windows, targets = create_user_specific_windows_and_targets(data_df, 'mood', n)

In [34]:
# Function to split data into train and test sets
def train_test_split(data, train_size=0.8):
    split_index = int(len(data) * train_size)
    train_data = data[:split_index]
    test_data = data[split_index:]
    return train_data, test_data



In [42]:
# Apply the train-test split to both windows and targets
train_windows, test_windows = train_test_split(windows, 0.95)
train_targets, test_targets = train_test_split(targets, 0.95)


In [38]:
# Avoid leakage
del test_windows[0]
del test_targets[0]